In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
import sys
import shutil

In [7]:

class CFG:
    NUM_EPOCHS = 1
    BATCH_SIZE = 2
    DROPOUT = 0.05
    # MODEL_NAME = "../llama3/Meta-Llama-3-8B/"
    MODEL_NAME = "C:/Users/jryhu/Documents/Git/llama-3-transformers-8b-chat-hf-v1/"
    SEED = 2024
    MAX_LENGTH = 128 # truncate the input to save memory, toy implementation only
    NUM_WARMUP_STEPS = 4 # toy implementation. 
    LR_MAX = 5E-5
    NUM_CLASS_LLAMA = 128 # with the llama model, we hope it can generate 128 features, which are combined with tfidf features.
    NUM_LABELS = 3 # The final number of labels
    LORA_RANK = 1 # Toy implementation 
    LORA_ALPHA = 2 # toy implementation
    LORA_MODULES = ['o_proj', 'v_proj']

if torch.cuda.is_available():
    DEVICE = 'cuda'
    print('GPU is used')
else:
    DEVICE = 'cpu'
    print('CPU is used')

GPU is used


In [8]:
# AGENT_PATH = "./agent/"

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(CFG.MODEL_NAME, torch_dtype = torch.bfloat16, device_map = "auto")
id_eot = tokenizer.convert_tokens_to_ids(["<|eot_id|>"])[0]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


In [21]:
def generate_answer(template):
    inp_ids = tokenizer(template, return_tensors="pt").to("cuda")
    out_ids = model.generate(**inp_ids,max_new_tokens=15).squeeze()
    start_gen = inp_ids.input_ids.shape[1]
    out_ids = out_ids[start_gen:]
    if id_eot in out_ids:
        stop = out_ids.tolist().index(id_eot)
        out = tokenizer.decode(out_ids[:stop])
    else:
        out = tokenizer.decode(out_ids)
    return out

In [27]:
class asker():
    def __init__(self):
        sys_prompt = """You are a helpful AI assistant, and your are very smart in playing 20 questions game,
                the user is going to think of a word, it can be only one of the following 3 categories:
                1. a place
                2. a person
                3. a thing
                So focus your area of search on these options. and give smart questions that narrows down the search space\n"""
        ask_prompt = sys_prompt + """your role is to find the word by asking him up to 20 questions, your questions to be valid must have only a 'yes' or 'no' answer.
                    to help you, here's an example of how it should work assuming that the keyword is Morocco:
                    examle:
                    <you: is it a place?
                    user: yes
                    you: is it in europe?
                    user: no
                    you: is it in africa?
                    user: yes
                    you: do most people living there have dark skin?
                    user: no
                    user: is it a country name starting by m ?
                    you: yes
                    you: is it Morocco?
                    user: yes.>

                    the user has chosen the word, ask your first question!
                    please be short and not verbose, give only one question, no extra word!\n\n"""
        chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{ask_prompt}<|eot_id|>\n\n"""
        chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        self.chat_template_root = chat_template
        self.chat_template = chat_template
    def append_history(self, question = None, answer = None, n=5):
        chat_template = self.chat_template_root
        ### only keep the most recent 5 questions
        if len(question)>n:
            for i in range(len(question)-n,len(question)):
                chat_template += f"{question[i]}<|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n\n"
                chat_template += f"{answer[i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        else:
            for q,a in zip(question, answer):
                chat_template += f"{q}<|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n\n"
                chat_template += f"{a}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
        self.chat_template = chat_template
        
class answerer():
    def __init__(self):
        self.sys_prompt = """You are a helpful AI assistant, and your are very smart in playing 20 questions game,
                you are going to think of a word, it can be only one of the following 3 categories:
                1. a place
                2. a person
                3. a thing
                Based on this word, you will respond "Yes", "No", or "I don't know" to user's question. Remember, you can only respond "Yes", "No", or "I don't know".\n"""        
        
#         chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{create_word_prompt}<|eot_id|>"""
#         chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        self.chat_template = None # chat_template
    def Create_Answer(self):
        self.create_word_prompt = self.sys_prompt + """Now, as a first step, please generate a single random word as the answer this this game. It can be a place, a person, or a thing. 
                    example 1:
                    <Cat.> \n
                    example 2:
                    <Bill Gates.> \n
                    
                    Tell me only one word. No extra word!\n"""
        chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{self.create_word_prompt}<|eot_id|>"""
        chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        self.chat_template = chat_template
        
    def Answer(self, question = None, answer = None):
        self.answer_prompt = self.sys_prompt + f"""
                    Here is an example of how it should work assume that the answer is Morocco:
                    examle:
                    <user: is it a place?
                    you: yes
                    user: is it in europe?
                    you: no
                    user: is it in africa?
                    you: yes
                    user: do most people living there have dark skin?
                    you: no
                    user: is it a country name starting by m ?
                    you: yes
                    user: is it Morocco?
                    yes: yes.>

                    Your answer is {answer}. User's question is {question}.
                    Now give me your respond. You can only respond "Yes", "No", or "I don't know"! No other words!"""
        chat_template = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{self.answer_prompt}<|eot_id|>"""
        chat_template += "<|start_header_id|>assistant<|end_header_id|>\n\n"
        self.chat_template = chat_template
        

## Answer Example

In [28]:
ans = answerer()
ans.Create_Answer()
answer_word = generate_answer(ans.chat_template)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [24]:
print(answer_word)

Paris


In [25]:
ans.Answer(question = "Is this in Spain?", answer = answer_word)
out = generate_answer(ans.chat_template)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [26]:
print(out)

No


## Asker Example

In [29]:
ask = asker()
question = generate_answer(ask.chat_template)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [31]:
print(question)
question_list = []
answer_list = []

Is it a place?


In [32]:
question_list.append(question)
answer_list.append("Yes")
ask.append_history(question = question_list, answer=answer_list)

In [33]:
print(ask.chat_template)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant, and your are very smart in playing 20 questions game,
                the user is going to think of a word, it can be only one of the following 3 categories:
                1. a place
                2. a person
                3. a thing
                So focus your area of search on these options. and give smart questions that narrows down the search space
your role is to find the word by asking him up to 20 questions, your questions to be valid must have only a 'yes' or 'no' answer.
                    to help you, here's an example of how it should work assuming that the keyword is Morocco:
                    examle:
                    <you: is it a place?
                    user: yes
                    you: is it in europe?
                    user: no
                    you: is it in africa?
                    user: yes
                    you: do most people living there have dar